<a href="https://colab.research.google.com/github/ParkhMinjun/python/blob/main/%ED%98%BC%EA%B3%B5%EB%B6%84/11.%EB%8F%84%EC%84%9C%20%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터베이스

## 파이썬에서 sql 사용:SQLite

In [ ]:
# 최신 sqlalchemy는 판다스에서 에러를 일으킵니다. 1.4.* 버전을 사용해 주세요. (https://github.com/pandas-dev/pandas/issues/40686)
!pip install -U sqlalchemy==1.4.46

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.31
    Uninstalling SQLAlchemy-2.0.31:
      Successfully uninstalled SQLAlchemy-2.0.31
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.46 which is incompatible.


In [ ]:
import sqlite3 # 패키지 임포

In [ ]:
conn = sqlite3.connect('ns_lib.db') # 데이터베이스 연결 , 해당 이름의 데이터베이스가 없다면 자동생성 해준다.

In [ ]:
import gdown

gdown.download('https://bit.ly/3RhoNho', 'ns_202104.csv', quiet=False)

Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:01<00:00, 52.5MB/s]


'ns_202104.csv'

In [ ]:
# 데이터베이스에 데이터 저장하려면 테이블이 필요
# CREATE TABLE nslib_book(name TEXT, author TEXT, borrow_count INTEGER) 소괄호 안에 만들고 싶은 열 나열 이때 데이터 타입 지

In [ ]:
# sql문 실행 cursor()-커넥션 객체 만듦,execute()
c = conn.cursor()

c.execute("CREATE TABLE nslib_book \
          (name TEXT, author TEXT, borrow_count INTEGER)")

In [ ]:
# c.execute("CREATE TABLE IF NOT EXISTS nslib_book \
#           (name TEXT, author TEXT, borrow_count INTEGER)") 기존 테이블이 없는경우 테이블 만듦

In [ ]:
# c.execute("DROP TABLE nslib_book") 기존 테이블 삭제

## 데이터프레임 데이터를 테이블에 추가

In [ ]:
import pandas as pd

ns_df = pd.read_csv('ns_202104.csv', low_memory=False)
ns_df.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
2,3,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,블랙피쉬,2021,9788968332982,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
3,4,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",문학세계사,2021,9788970759906,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
4,5,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,김영사,2021,9788934990833,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN


In [ ]:
# 한 행씩 반복
for index, row in ns_df.iterrows(): # iterrows() 데이터프레임의 인덱스와 행 객체 반
    c.execute("INSERT INTO nslib_book (name,author,borrow_count) \
              VALUES (?,?,?)", (row['도서명'], row['저자'], row['대출건수']))

In [ ]:
for index, row in ns_df.iterrows():
    pass

In [ ]:
# sql 문 없이 테이블 추가:to_sql()매서드 - 더 효율적이다.


book_df = ns_df[['도서명','저자','대출건수']]
book_df.head()

,도서명,저자,대출건수
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [ ]:
book_df.columns = ['name', 'author', 'borrow_count'] # 열 이름 통일을 위해 이름 바꿈
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [ ]:
book_df.to_sql('nslib_book', conn, if_exists='replace', index=False) # 첫번째 매개변수 사용할 테이블 이름,두번째 커넥션 객체

401682

## 테이블 데이터 읽기

In [ ]:
c.execute("SELECT * FROM nslib_book")

In [ ]:
# 열 반환
c.fetchone()

('인공지능과 흙', '김동훈 지음', 0)

In [ ]:
# 한번더 호출 행 반환
c.fetchone()

('가짜 행복 권하는 사회', '김태형 지음', 0)

In [ ]:
# 한개 이상의 행 반환
c.fetchmany(3)

[('나도 한 문장 잘 쓰면 바랄 게 없겠네', '김선영 지음', 0),
 ('예루살렘 해변', '이도 게펜 지음, 임재희 옮김', 0),
 ('김성곤의 중국한시기행 : 장강·황하 편', '김성곤 지음', 0)]

In [ ]:
all_rows = c.fetchall() # 전체 출력

In [ ]:
book_df = pd.DataFrame(all_rows) # 데이터프레임 만들
book_df.head()

,0,1,2
0,처음 읽는 음식의 세계사,"미야자키 마사카츠 지음, 한세희 옮김",0
1,아르센 벵거 자서전 My Life in Red and White,"아르센 벵거 지음, 이성모 옮김",0
2,쓰고 달콤한 직업,천운영 지음,0
3,일잘러의 노션 100% 활용법,제레미 강 지음,0
4,타이난 골목 노포 산책,"천구이팡 지음, 심혜경 외 옮김",0


In [ ]:
# 처음부터 데이터 프레임 만들기:rea_sql_query()
book_df = pd.read_sql_query("SELECT * FROM nslib_book", conn)
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [ ]:
book_df = pd.read_sql_table('nslib_book', 'sqlite:///ns_lib.db') # 테이블 이름만 전달, 두번째 매개변수는 데이터베이스 종류와 이름 :///문자로 연결
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


## 통계량 구하기

In [ ]:
len(book_df)

401682

In [ ]:
# 데이터프레임 만들 필요없이 간단한 통계

In [ ]:
c.execute("SELECT count(*) FROM nslib_book") # 행 갯수
c.fetchone()

(401682,)

In [ ]:
c.execute("SELECT sum(borrow_count) FROM nslib_book") # 덧셈
c.fetchone()

(4400145,)

In [ ]:
c.execute("SELECT avg(borrow_count) FROM nslib_book") # 평균
c.fetchone()

(10.95429966988812,)

## 테이블 데이터 정렬

In [ ]:
# ORDER BY 기본적으로 오름차순 DESC로 내림차순 변경
# LIMIT 수 제한
# 가장 대출건수가 많은 10개의 도서 출력
c.execute("SELECT * FROM nslib_book ORDER BY borrow_count DESC LIMIT 10")
c.fetchall()

[('사피엔스 :유인원에서 사이보그까지, 인간 역사의 대담하고 위대한 질문 ', '유발 하라리 지음 ;조현욱 옮김', 1468),
 ('해커스 토익:Listening', 'David Cho 지음', 1065),
 ('7년의 밤 :정유정 장편소설 ', '정유정 저', 683),
 ('냉정과 열정사이:Blu', '츠지 히토나리 지음;양억관 옮김', 524),
 ('남한산성:김훈 장편소설', '김훈 지음', 501),
 ('해리포터와 혼혈왕자', '조앤 K. 롤링 지음;최인자 옮김', 451),
 ('해커스 토익:Listening', 'David Cho 지음', 440),
 ('다빈치 코드', '댄 브라운 지음;양선아 옮김', 440),
 ('신:베르나르 베르베르 장편소설', '베르나르 베르베르 지음;이세욱 옮김', 432),
 ('경제학 콘서트', '팀 하포드 지음;김명철 옮김', 425)]

In [ ]:
# 프로그램이나 노트북이 종료되면 연결이 자동종료되지만 데이터베이스 사용을 마칠때 커서와 커넥션 객체를 명시적으로 닫아주는 것이 좋다.

c.close()
conn.close()